In [ ]:
import numpy as np
from scipy.special import factorial, comb

In [2]:
def evaluate_energy(solution, Q, const):
    return const + np.dot(solution, np.dot(Q, solution))

def idx_onehot(i, p, P):
    return i*P + p

def symmetrize(Q):
    return (Q + Q.T)/2

def build_pen_TSP(Nc):
    H = np.zeros((Nc**2, Nc**2))
    for t in range(Nc):
        for i in range(Nc):
            H[idx_onehot(t, i, Nc), idx_onehot(t, i, Nc)] = -2
            for i_prime in range(Nc):
                if i_prime != i:
                    H[idx_onehot(t, i, Nc), idx_onehot(t, i_prime, Nc)] += 1
            for t_prime in range(Nc):
                if t_prime != t:
                    H[idx_onehot(t, i, Nc), idx_onehot(t_prime, i, Nc)] += 1
    const = 2*Nc
    H = symmetrize(H)
    return H, const

def state_inttobin(i, n):
    state_string = bin(i)[2:].zfill(n)
    return [int(b) for b in state_string]

def state_bintoint(b):
    b_numb = ''.join([str(k) for k in b])
    return int(b_numb, 2)

def state_syndrome(x, N):
    parity = np.sum(x) - N
    row_syn, col_syn = [], []
    for i in range(N):
        if np.sum(x[i]) != 1:
            row_syn.append(i)
        if np.sum(x[:, i]) != 1:
            col_syn.append(i)
    return parity, row_syn, col_syn

def pos_ones(x, row_list, iscol = False):
    col_ones = []
    for r in row_list:
        if iscol:
            vec = x[:,r]
        else:
            vec = x[r]
        col_ones.append( np.where(vec == 1)[0] )
    return col_ones

In [ ]:
N = 4
v_test = 7

Q, const = build_pen_TSP(N)
states = []
for i in range(2**(N**2)):
    v = evaluate_energy(state_inttobin(i, N**2), Q, const)
    if v == v_test:
        states.append(np.reshape(state_inttobin(i, N**2), newshape = (N, N)))



states_zero = []
states_plusone = []
states_minusone = []
states_plustwo = []
states_minustwo = []
states_plusthr = []
states_minusthr = []

# n_m3 = factorial(N)*comb(N,3)/6
# n_m2 = 2*comb(N,4)*factorial(N)
# n_m1 = factorial(N)*comb(N,4)*(N-4) + factorial(N)*N*comb(N,4)*3/2
# n_z = 5*factorial(N)*comb(N,6) + 3*factorial(N)*comb(N,4)*(comb(N-4,2)+4*(N-3)) + factorial(N)*comb(N,3)
# n_p1 = factorial(N)*5*comb(N,5)*((N+6)*(N-5)/4+6) + factorial(N)*3/2*comb(N,3)*(1+4*(N-3)**2+3/8*comb(N-3,3)*(N+10)) + factorial(N)*comb(N,2)*(N-2)*(N+3)/3
# n_p2_old = factorial(N)*6*comb(N,4)*(5+(N-4)/2*(7*N-19+(N-5)/3*(N+17)/8)) + factorial(N)*comb(N,2)*(N-2)*(2+(N-3)/12*(N+8))
# n_p2 = factorial(N)*6*comb(N,4)*(5+8*(N-4)+7*comb(N-4,2)+3*comb(N-4,3)+comb(N-4,4)/2) + factorial(N)*comb(N,2)*(N-2)*(2+(N-3)/12*(N+8))
# n_p3 = factorial(N)*comb(N,3)/2*(2+15/4*(N-3)*N+36*comb(N-3,2)+41*comb(N-3,3)+63/2*comb(N-3,4)+5*comb(N-3,5)*(N+28)/8)

tot = factorial(N)**2/factorial(N-3)/2*( 1 + (N-1)*(N+5)/4 + 1/(N-2) + 2*(N-2) + (N-3)*(N-4)*(1+(N-5)/16)/2 )
short = factorial(N)*(comb(N,2)+21*comb(N,3)+57*comb(N,4)+45*comb(N,5)+45/4*comb(N,6))

print(f"Violation {v_test} states: {len(states)}, theo {int(tot)}")
print(f"Violation {v_test} states: {len(states)}, theo {int(short)}")
print()

# r = factorial(N)*comb(N,3)*( 47/3+25*(N-3)+137/4*comb(N-3,2)+59/2*comb(N-3,3)+18*comb(N-3,4)+45/2*comb(N-3,5)+15/4*comb(N-3,6) )
# b = factorial(N)*comb(N,4)*( 50+125/2*(N-4)+45*comb(N-4,2)+18*comb(N-4,3)+3*comb(N-4,4) )
# g = factorial(N)*comb(N,5)*( 30+95/6*(N-5)+5/2*comb(N-5,2) )
# print(f"Violation {v_test} states: {len(states)}, compared with theor {int( r + b + g )}")
# print()


for st in states:
    parity, row_syn, col_syn = state_syndrome(st, N)
    match parity:
        case 0:
            states_zero.append(st)
        case 1:
            states_plusone.append(st)
        case -1:
            states_minusone.append(st)
        case 2:
            states_plustwo.append(st)
        case -2:
            states_minustwo.append(st)
        case 3:
            states_plusthr.append(st)
        case -3:
            states_minusthr.append(st)
        case _:
            print(parity)
            raise ValueError("Discrepancy at least 3")

# p_m3 = factorial(N)*comb(N,3)/6
# p_m2 = factorial(N)*comb(N,4)*2
# p_m1 = factorial(N)*comb(N,4)*( 6 + 5/2*(N-4) )
# p_z = factorial(N)*comb(N,3) + factorial(N)*comb(N,6)*5 + factorial(N)*comb(N,4)*3*( 4 + 4*(N-4) + comb(N-4,2) )
# p_p1 = factorial(N)*comb(N,5)*5*(6 + (N-5)*(N+6)/4) + factorial(N)*comb(N,3)*(15/2+7*(N-3)+12*comb(N-3,2)+9*comb(N-3,3)+9/4*comb(N-3,4)) 
# p_p2 = factorial(N)*comb(N,3)*3*(2+(N-3)+comb(N-3,2)/6) + factorial(N)*comb(N,4)*6*(5+8*(N-4)+191/24*comb(N-4,2)+comb(N-4,3)/8)
# p_p3 = factorial(N)*comb(N,3)/2*( 2+15*(N-3)+87/2*comb(N-3,2)+41*comb(N-3,3)+63/2*comb(N-3,4)+45/2*comb(N-3,5)+15/4*comb(N-3,6) )

# print(f"{len(states_minusthr)} states with -3 parity, compared with theor, {int(p_m3)}, {int( factorial(N)*comb(N,3)/6 )}")
# print(f"{len(states_minustwo)} states with -2 parity, compared with theor, {int(p_m2)}, {int( 2*comb(N,4)*factorial(N) )}")
# print(f"{len(states_minusone)} states with -1 parity, compared with theor, {int(p_m1)}, {int( factorial(N)*comb(N,4)*(N-4)+factorial(N)*N*comb(N,4)*3/2 )}")
# print(f"{len(states_zero)} states with 0 parity, compared with theor, {int(p_z)}, {int( 5*factorial(N)*comb(N,6)+3*factorial(N)*comb(N,4)*(comb(N-4,2)+4*(N-3))+factorial(N)*comb(N,3) )}")
# print(f"{len(states_plusone)} states with +1 parity, compared with theor, {int(p_p1)}, {int( factorial(N)*5*comb(N,5)*((N+6)*(N-5)/4 + 6) + factorial(N)/2*N*comb(N-1,2)*(1+(N-3)*N+comb(N-3,2)*(6+(N-5)*(N+10)/8)) + factorial(N)*comb(N,2)*(N-2)*(N+3)/3 )}")
# print(f"{len(states_plustwo)} states with +2 parity, compared with theor, {int(p_p2)}, {int( factorial(N)*comb(N,2)*comb(N-2,2)*(  5 + (N-4)/2*(7*N-19 + (N-5)/3*(N+17)/8) )  + np.rint( factorial(N)*comb(N,2)*(N-2)*(2+(N-3)/12*(N+8)) ) )}")
# print(f"{len(states_plusthr)} states with +3 parity, compared with theor, {int(p_p3)} {int( factorial(N)*comb(N,3)/2 * (  2 + 15/4*(N-3)*N + 36*comb(N-3,2) + 41*comb(N-3,3) + 63/2*comb(N-3,4) + 5*comb(N-3,5)*(N+28)/8  ))}")
# print()

# p_m3_z = factorial(N)*comb(N,3)*7/6 + factorial(N)*comb(N,6)*5 + factorial(N)*comb(N,4)*( 20+29/2*(N-4)+3*comb(N-4,2) )  
# p_p1_p3 = factorial(N)*comb(N,5)*5*(6+3*(N-5)+comb(N-5,2)/2) + factorial(N)*comb(N,3)*(29/2+35/2*(N-3)+137/4*comb(N-3,2)+59/2*comb(N-3,3)+18*comb(N-3,4)+45/4*comb(N-3,5)+15/8*comb(N-3,6)) + factorial(N)*comb(N,4)*6*(5+8*(N-4)+191/24*comb(N-4,2)+comb(N-4,3)/8) 
# print(f"{len(states_minusthr)+len(states_minustwo)+len(states_minusone)+len(states_zero)} states with parity from -3 till 0, compared with theor, {int(p_m3_z)}")
# print(f"{len(states_plusthr)+len(states_plustwo)+len(states_plusone)} states with parity from 1 till 3, compared with theor, {int(p_p1_p3)}")

# comp,ladd = [],[]
# for st in states_plusthr:
#     parity, row_syn, col_syn = state_syndrome(st, N)

#     ones_r = pos_ones(st, row_syn, iscol = False)
#     r_set = set()
#     for ones in ones_r:
#         r_set.update(list(ones))

#     ones_c = pos_ones(st, col_syn, iscol = True)
#     c_set = set()
#     for ones in ones_r:
#         c_set.update(ones)

#     if len(r_set) == 3 and len(c_set) == 3:
#         comp.append(st)
#     elif len(r_set) == 4 and len(c_set) == 4:
#         ladd.append(st)
#     else:
#         print(len(row_syn), len(col_syn))
#         print("Error")


# print(f"{len(comp)} states with 3 parity, compact type, compared with theor {int( comb(N,3)*(  comb(N,2)*2*(N-2)*factorial(N-3)) )}")
# print(f"{len(ladd)} states with 3 parity, ladder type, compared with theor {int( comb(N,3)*3*comb(N,2)*2*(N-2)*(N-3)*( 2*(N-3)*factorial(N-4))  )}")


Violation 7 states: 0, theo 3528
Violation 7 states: 0, theo 3528



In [19]:

pred = [816, 45300]
for i, N in enumerate([4,5]):
    full = comb(N,3)*(  comb(N,2)*2*(N-2)*factorial(N-3) + 3*comb(N,2)*2*(N-2)*(N-3)*( 2*(N-3)*factorial(N-4) + (N-4)*comb(N-3,2)*factorial(N-5) )       +3*comb(N,2)*comb(N-2,2)*( 2*(N-3)*factorial(N-4) + (N-4)*comb(N-3,2)*factorial(N-5) )      + 3*2*comb(N,2)*(N-2)*comb(N-3,2)*( comb(N-3,2)*factorial(N-5)*12 + 4*(N-3)*(N-5)*comb(N-4,2)*factorial(N-6) + comb(N-5,2)*comb(N-3,4)*comb(4,2)*factorial(N-7) ) + comb(N,6)*comb(6,2)*comb(4,2)*( comb(N-3,3)*comb(6,3)*factorial(N-6) + comb(N-3,2)*comb(6,2)*(N-6)*comb(N-5,2)*factorial(N-7) + 6*(N-3)*comb(N-6,2)*comb(N-4,4)*comb(4,2)*factorial(N-8) + comb(N-6,3)*comb(N-3,2)*comb(N-5,2)*comb(N-7,2)*factorial(N-9)  )  )
    full2 = comb(N,3)*factorial(N)/2 * (  2 + 15/4*(N-3)*N  +  6*comb(N-3,2)*( 6+2*(N-5)+comb(N-5,2)/4 )  + (N-3)/24*comb(N-4,2)*( 40 + 45*(N-6) + 18*comb(N-6,2)+ 3/2*comb(N-6,3) )   )
    fin = comb(N,3)*factorial(N)/2 * (  2 + 15/4*(N-3)*N + 36*comb(N-3,2) + 41*comb(N-3,3) + 63/2*comb(N-3,4) + 5*comb(N-3,5)*(N+28)/8  )
    print(f"{pred[i]} states with 3 parity, 33 type, compared with theor {int(np.rint(fin))}")

    print()

816 states with 3 parity, 33 type, compared with theor 816

45300 states with 3 parity, 33 type, compared with theor 45300

